# 1. Import Modules

In [1]:
from bs4 import BeautifulSoup
import requests
import time
import pandas as pd


# 2. Read Data

In [2]:
file_date = input("請輸入資料區間： ")

請輸入資料區間： 20240105_20240107


In [3]:
df_bw_file = pd.read_csv(f'BusinessWeekly_Line_{file_date}.csv')

In [4]:
df_bw_file.head()

,URL
0,https://www.businessweekly.com.tw/Archive/Arti...
1,https://www.businessweekly.com.tw/Archive/Arti...
2,https://bwnet.pse.is/5hrsk9
3,https://bwnet.pse.is/5hrtqk
4,https://bwnet.pse.is/5hry6l


### 說明：
商周的網頁目前有兩種形式，如果是線上讀，會有'https://www.businessweekly.com.tw/Archive/'
在網址裡面，故要另外抓取資料

In [5]:
url_number = 0

content_id = [] # 這邊不會用到
df_bw_sw_all = pd.DataFrame()

bw_column_name = ['content_id'
                 ,'content_title_h1'
                 ,'content_title_description'
                 ,'content_class_main'
                 ,'content_class_sub'
                 ,'content_date_time'
                 ,'content_tag']

for URL in df_bw_file['URL']:
    
    content_id = [] # 基本上不會用到，如果是舊版網頁，此欄位先放文章的url，以便之後查詢
    
    try:
        time.sleep(5)
        print("original url:", URL)
        headers={"user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36"}
        response = requests.get(URL, headers = headers)
        bw_html = BeautifulSoup(response.text, "lxml")
            
        try:
            url = bw_html.find("meta", {"property":"og:url"}).get("content")

        except:
            url = bw_html.find("meta", {"name":"og:url"}).get("content")
            
        
        print("full url:", url)
        
        # --- Begin 大標題 ---
        lst_title_h1 = []
        
        if '/Archive/' in url:
            
            title_h1 = bw_html.find("meta", {"name":"title"}).get("content")
            print("title: ", title_h1)
            lst_title_h1.append(title_h1)
            
        elif "alive.businessweekly" in url:
            
            title_h1 = bw_html.find("meta", {"property":"og:title"}).get("content")
            print("alive_title: ", title_h1)
            lst_title_h1.append(title_h1)
        
        else:
            
            title_h1 = bw_html.find_all("h1")  # 搜尋所有h1
            
            for title in title_h1:
                print('title:', title.getText())  # 只提取文字部分
                lst_title_h1.append(title.getText())

        # --- End 大標題 ---
        
        # --- Begin 描述 ---
        
        ## 改成 meta description

        lst_title_description = []
        
        title_description = bw_html.find("meta", {"name":"description"}).get("content")
        print("title description: ", title_description)
        lst_title_description.append(title_description)

        # --- End 描述 ---
     
        # --- Begin 文章分類 Main and sub ---
        
        lst_content_class_main = []
        lst_content_class_sub = []
        
        if '/Archive/' in url:
            
            content_class_main = bw_html.find("div", {"class":"author infoList"}).getText().replace("｜","").split('\n')[1].lstrip()
            print('main category:', content_class_main)
            lst_content_class_main.append(content_class_main)

            lst_content_class_sub.append(" ")
        
        elif "alive.businessweekly" in url:
            
            article_h2 = bw_html.find_all("h2")  # 搜尋所有h2
            
            h2_num = 1
            
            for h2 in article_h2:
                print('alive_class:', h2.getText())  # 只提取文字部分
                
                if h2_num == 1:
                    lst_content_class_main.append(h2.getText())
                    lst_content_class_sub.append(" ")
                
                h2_num += 1

        else:
            content_class_main = bw_html.find("meta", {"property":"article:section"}).get("content")
            print('main category:', content_class_main)
            lst_content_class_main.append(content_class_main)

            content_class_sub = bw_html.find("meta", {"itemprop":"articleSection"}).get("content")
            print('sub category:', content_class_sub)
            lst_content_class_sub.append(content_class_sub)

        # --- End 文章分類 Main and sub ---
        
        # --- Begin 文章發佈日 ---

        lst_content_date = []
        
        if '/Archive/' in url:

            ct_date  = bw_html.find("span", {"itemprop":"datePublished"}).getText()  # 文章發佈日
            print("文章發布日:", ct_date)
            lst_content_date.append(ct_date)
        
        elif "alive.businessweekly" in url:
            
            # Find the div with class "annotation"
            annotation_div = bw_html.find('div', class_='annotation')
            
            # Find the paragraph with class "text-grey" inside the annotation div
            date_paragraph = annotation_div.find('p', class_='text-grey')

            # Extract the text content (date)
            ct_date = date_paragraph.get_text(strip=True)
            
            print("alive_文章發布日:", ct_date)
            lst_content_date.append(ct_date)
        
        else:
            
            ct_date  = bw_html.find("div", {"id":"AppGetDate"})  # 文章發佈日
            lst_content_date.append(ct_date.getText()[:10])
            print("文章發布日:", ct_date.getText()[:10])

        # --- End 文章分類及文章發佈日 ---
        
        # --- Begin 標籤 ---

        lst_content_tag = []
        
        if "alive.businessweekly" in url:
            
            content_tag = bw_html.find("input", {"name":"gtm_keywords_hidden"}).get('value')  # 標籤
            print('alive_文章標籤:', content_tag)
            lst_content_tag.append(content_tag)
            
        else:
        
            content_tag = bw_html.find("meta", {"name":"keywords"}).get('content')  # 標籤
            print('文章標籤:', content_tag)
            lst_content_tag.append(content_tag)
        
        # --- End 標籤 ---
          
        # List 轉 Series
        series_id = pd.Series(content_id)
        series_title_h1 = pd.Series(lst_title_h1)
        series_title_description = pd.Series(lst_title_description)
        series_content_class_main = pd.Series(lst_content_class_main)
        series_content_class_sub = pd.Series(lst_content_class_sub)
        series_content_date = pd.Series(lst_content_date)

        lst_content_tag = ",".join(lst_content_tag) # 把list中的資料用","join起來
        series_content_tag = pd.Series(lst_content_tag)
        
        # Series 放到 DataFrame
        df_bw_sw_keyword = pd.concat([series_id
                                     ,series_title_h1
                                     ,series_title_description
                                     ,series_content_class_main
                                     ,series_content_class_sub
                                     ,series_content_date
                                     ,series_content_tag]
                                     ,axis = 1)
        
        df_bw_sw_keyword.columns = bw_column_name

        url_number += 1
        
        df_bw_sw_all = pd.concat([df_bw_sw_all, df_bw_sw_keyword], ignore_index=True)
        
    except:
        print("Manul:", url)
        
        content_id.append(url)
        print("content_id", content_id)
        
        series_id = pd.Series(content_id)
        
        df_bw_sw_keyword_except = pd.concat([series_id], axis = 1)
        
        df_bw_sw_keyword_except.columns = ["content_id"]
        df_bw_sw_all = pd.concat([df_bw_sw_all, df_bw_sw_keyword_except], ignore_index=True)
        
        url_number += 1
    
print("----------------- Finish -------------------")

df_bw_sw_all.to_csv(f'{file_date}_BusinessWeekly_LINE.csv', index = False, encoding = 'UTF-8-sig')


original url: https://www.businessweekly.com.tw/Archive/Article?StrId=7009464&utm_source=Line&utm_medium=social&utm_content=bw&utm_campaign=content&Ftag=7009464
full url: https://www.businessweekly.com.tw/Archive/Article?StrId=7009464
title:  信義房屋接班人的AI轉型計　讓成交率多2成、離職降1成！ - 商業周刊第1886期 - 商周線上讀
title description:  信義房屋辦人周俊吉的大兒子周耕宇，5年前接下數位轉型重任，靠著不強制全員配合、翻轉使命必達等傳統思維，在疫情、房市交易量萎縮之際，拚出一個成交轉換率高2成的AI新兵器，還靠各種數位轉型，穩住企業獲利，順利接班成為新董座。
main category: 產業風雲
文章發布日: 2024-01-04
文章標籤: AI,數位轉型,周耕宇,信義房屋,房仲


C:\Users\JoeJian\AppData\Local\Temp\ipykernel_39768\134116846.py:161: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  series_id = pd.Series(content_id)


original url: https://www.businessweekly.com.tw/Archive/Article?StrId=7009455&utm_source=Line&utm_medium=social&utm_content=bw&utm_campaign=content&Ftag=7009455
full url: https://www.businessweekly.com.tw/Archive/Article?StrId=7009455
title:  一年股價漲1.9倍僅次輝達　Meta佐伯格認錯逆轉危機 - 商業周刊第1886期 - 商周線上讀
title description:  在2023年，臉書母公司Meta經歷逆轉勝，股價上漲1.9倍，超越蘋果、微軟，在標準普爾500大企業中僅次於輝達。Meta執行長佐伯格實施裁員和財務改善，積極回應投資者擔憂，並轉變態度，開始聆聽股東意見，將公司聚焦在當下重要事務。Meta透過人工智慧、開源系統提升效率，成功應對市場變化。儘管面臨挑戰，分析師仍看好其未來發展。
main category: 投資焦點
文章發布日: 2024-01-04
文章標籤: Meta,臉書,佐伯格,元宇宙,短影音,Threads,人工智慧


C:\Users\JoeJian\AppData\Local\Temp\ipykernel_39768\134116846.py:161: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  series_id = pd.Series(content_id)


original url: https://bwnet.pse.is/5hrsk9
full url: https://www.businessweekly.com.tw/international/blog/3014408
title: 全球航線大亂，航運股強勁漲停⋯收一櫃賺一櫃趁現在
title description:  紅海危機造成全球海運航線大亂，貨櫃運價從疫後的谷底攀升，迄今漲了3~4倍。除了航商率先受惠，全球貿易需求的外溢效應爆開，讓散裝、航空、貨攬、相關業者全都得利。缺船、缺櫃的「供應鏈焦慮」大爆發，航運股成
main category: 國際
sub category: 全球話題
文章發布日: 2024/01/05
文章標籤:  長榮,紅海, 航運股, 運價


...

C:\Users\JoeJian\AppData\Local\Temp\ipykernel_39768\134116846.py:161: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  series_id = pd.Series(content_id)


original url: https://bwnet.pse.is/5hwg4e
full url: https://www.businessweekly.com.tw/careers/blog/3014412
title: 出國旅行遇大雨⋯計程車司機一句話，教會我什麼叫正向思考
title description:  心態，是你選擇看待事情的角度，就算是一樣的事物，每個人看到的也會有所不同。小小的心態差異，就會決定我們是否能主導自己的人生，或是受到環境支配；會決定我們能否產生動力和決心，或因位阻力和擔憂變得軟弱。
main category: 職場
sub category: 心靈成長
文章發布日: 2024/01/05
文章標籤: 司機,成功,心態,計程車,旅遊


C:\Users\JoeJian\AppData\Local\Temp\ipykernel_39768\134116846.py:161: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  series_id = pd.Series(content_id)


original url: https://bwnet.pse.is/5hwgn8
full url: https://www.businessweekly.com.tw/careers/blog/3014413
title: 一份讓老闆秒懂重點的「進度報告」怎麼寫？
title description:  管理偌大的團隊，每天日理萬機，主管關切的，不是我們每天「已經做」的流水帳，這就是我們的本份。他更仰賴我們，能提出業務上有哪些「應該做」、甚至是「做更多」的部分。找到之所以現在「還沒做」的癥結，有效動員
main category: 職場
sub category: 職場修練
文章發布日: 2024/01/05
文章標籤: 進度報告,職場,向上管理,主管,溝通技巧
----------------- Finish -------------------


C:\Users\JoeJian\AppData\Local\Temp\ipykernel_39768\134116846.py:161: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  series_id = pd.Series(content_id)
